In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import yfinance as web
from IPython.display import display
from tkinter import *
from tkinter import Tk
from tkinter import ttk, Scrollbar, VERTICAL

'''
class Table: 
    def __init__(self, window, data): 
        self.tree = ttk.Treeview(window, show='headings')

        # Configurar as colunas
        cols = list(data.columns)
        self.tree['columns'] = cols
        for col in cols:
            if col == '':
                pass
            else:
                self.tree.column(col, width=100, anchor='center')
                self.tree.heading(col, text=col)

        # Adicionar os dados
        for i, row in data.iterrows():
            self.tree.insert(parent='', index='end', iid=i, text='', values=tuple(row))

        # Adicionar scroll vertical
        vsb = Scrollbar(window, orient=VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=vsb.set)
        vsb.pack(side='right', fill='y')

        self.tree.pack(expand=True)
'''

def ibovespa():
    cotacao_ibov = web.download('^BVSP', start='2020-01-01', end='2020-11-10')
    display(cotacao_ibov)
    # Criar a tabela
    # c = Table(frame_tab, data=cotacao_ibov)

    # Gráfico
    '''figura = plt.Figure(figsize=(10, 5))
    grafico = figura.add_subplot(111)
    grafico.plot(cotacao_ibov['Adj Close'])
    canva = FigureCanvasTkAgg(figura, frame_grf)
    canva.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)'''
    cotacao_ibov['Adj Close'].plot(figsize=(15,5))
    plt.title('Cotações')
    plt.show()
    plt.close()

    # Retorno Ibov
    retorno_ibov = cotacao_ibov['Adj Close'].iloc[-1] / cotacao_ibov['Adj Close'].iloc[0] - 1
    print(f'Retorno de {retorno_ibov:.2%}')

    # Retorno Ibov
    retorno_ibov = cotacao_ibov['Adj Close'].iloc[-1] / cotacao_ibov['Adj Close'].iloc[0] - 1
    print(f'Retorno de {retorno_ibov:.2%}')

    # Gráfico Média Móvel
    cotacao_ibov['Adj Close'].rolling(21).mean().plot(label='MM21')
    cotacao_ibov['Adj Close'].rolling(34).mean().plot(label='MM34')
    plt.title('Média Móvel')
    plt.show()
    plt.close()
    

def carteira_info():
    # Importar dados da carteira
    carteira = pd.read_excel('Carteira.xlsx')
    display(carteira)

    # Criar dataframe vazio
    cotacoes_carteira = pd.DataFrame()

    cotacoes_carteira = web.download('BOVA11.SA', start='2020-01-01', end='2020-11-10')
    # d = Table(window, cotacoes_carteira)
    plt.show()
    plt.close()

    # Definição linha de código de preenchimento de apenas 1 ativo
    cotacoes_carteira = pd.DataFrame()

    extracao = web.download('BOVA11.SA', start='2020-01-01', end='2020-11-10')
    cotacoes_carteira['BOVA11.SA'] = extracao ['Adj Close']
    display(cotacoes_carteira)


    # Preenchimento de vários ativos
    cotacoes_carteira = pd.DataFrame()

    for ativo in carteira['Ativos']:
        extracao = web.download(f'{ativo}.SA', start='2020-01-01', end='2020-11-10')
        cotacoes_carteira[ativo] = extracao['Adj Close']

    display(cotacoes_carteira)

    # Tratamento de problemas
    cotacoes_carteira = cotacoes_carteira.ffill()
    cotacoes_carteira.info()

    # Visualização de ativos no gráfico
    carteira_norm = cotacoes_carteira / cotacoes_carteira.iloc[0]
    carteira_norm.plot(figsize=(15, 5))
    plt.legend(loc='upper left')
    plt.title('Ativos')
    plt.show()
    plt.close()

def carteira_ib():
    carteira = pd.read_excel('Carteira.xlsx')
    cotacoes_carteira = pd.DataFrame()
    for ativo in carteira['Ativos']:
        extracao = web.download(f'{ativo}.SA', start='2020-01-01', end='2020-11-10')
        cotacoes_carteira[ativo] = extracao['Adj Close']

    # Coletando cotações do IBOVESPA
    cotacao_ibov = web.download('^BVSP', start='2020-01-01', end='2020-11-10')
    display(cotacao_ibov)

    # Valor investido
    valor_investido = pd.DataFrame()

    for ativo in carteira['Ativos']:
        valor_investido[ativo] = cotacoes_carteira[ativo] * carteira.loc[carteira['Ativos']==ativo, 'Qtde'].values[0]
    display(valor_investido)

    # Comparativo IBOVESPA com a carteira - Gráfico
    valor_investido['Total'] = valor_investido.sum(axis=1)

    valor_investido_norm = valor_investido / valor_investido.iloc[0]
    cotacao_ibov_norm = cotacao_ibov / cotacao_ibov.iloc[0]

    valor_investido_norm['Total'].plot(figsize=(15, 5), label='Carteira')
    cotacao_ibov_norm['Adj Close'].plot(label='IBOV')
    plt.legend()
    plt.title('Carteira x IBOVESPA')
    plt.show()
    plt.close()

    # Comparativo IBOVESPA com a carteira - Valores de Retorno

    retorno_carteira = valor_investido['Total'].iloc[-1] / valor_investido['Total'].iloc[0] - 1
    retorno_ibov = cotacao_ibov['Adj Close'].iloc[-1] / cotacao_ibov['Adj Close'].iloc[0] - 1
    print(f'Retorno da Carteira: {retorno_carteira:.2%}')
    print(f'Retorno IBOV: {retorno_ibov:.2%}')

    # Correlação
    correlacao = valor_investido['Total'].corr(cotacao_ibov['Adj Close'])
    print(F'Correlação: {correlacao}')

window = Tk()
window.title('Finanças')
window.geometry('800x800')

# Adicione o conteúdo aos botões
ibovespa_btn = ttk.Button(window, text='IBOVESPA', command=ibovespa)
ibovespa_btn.pack(pady=40)

carteira_btn = ttk.Button(window, text='Carteira', command=carteira_info)
carteira_btn.pack(pady=40)

carteira_ib_btn = ttk.Button(window, text='Carteira x IBOVESPA', command=carteira_ib)
carteira_ib_btn.pack(pady=40)

window.mainloop()